# Summer Data science project


### Introduciton
The goal of this probability and statistics project is aimed towards holistic growth of my abilities to work on a data science project from start to finish in hopes of replicating the process of a corporate job to better prepare me for the industry. The difference between this project and the projects ive worked on previously will be the emphasis on project planning, data collection, optimization, documentation and producitionization things I have previously ignored. 

## Table of Contents.
1. Project planning and research. 
2. Scraping data
3. Data cleaning
4. Exploratory analysis
5. model building
6. Putting model into production (producitionization)
7. documentation. 



## 1. Project planning and research.
#### Project Ideas

    1. Weightlifting
        -Personal Record classification model (not alot of accessible data online) Can use web scraping, (openpowerlifting provides good powerlifting data.) you can add more data through research of powerlifters youtube videos(youtube api). 
        -although not big of a runner myself maybe route reccomendations through (strava api)
    2. Music 
        -Music Reccomendation model  (spotify api)
    3. Reading 
        -book reccomendation model For Ellie( Google Books API, Open Library API)
    4. Video games
        -Video game stats, Match making rank models, champion patch analysis, 
    5. any project resulting from Data from US census 
        -https://www.census.gov/data/developers/data-sets.html



 

I created a list of activities I'm passionate about. Although each subject excites me, I found myself particularly drawn to developing a music recommendation model. This project resonates with me deeply because I frequently listen to music, and I have easy access to my own listening data through the spotify api. Additionally, the availability of APIs and online data for this area makes it a practical and engaging choice for my research.

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
import pandas as pd
import time
import requests
from requests.exceptions import RequestException
from spotipy.oauth2 import SpotifyClientCredentials
import time
import requests.exceptions
from spotipy.exceptions import SpotifyException


## Spotify API reserach


In [2]:

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="400f5690f0bd413aab3a0b258e0c41f4",
                                               client_secret="03086e1b011c4cc18297e8c7c1dff61a",
                                               redirect_uri="http://localhost:7777/callback",
                                               scope="user-library-read"))
results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

0 Coldplay  –  Hymn for the Weekend - Seeb Remix
1 Otto Knows  –  Back Where I Belong (feat. Avicii)
2 Calvin Harris  –  Under Control (feat. Hurts)
3 ISOxo  –  how2fly
4 Miguel  –  Girl With The Tattoo Enter.lewd
5 Hozier  –  Too Sweet
6 Franz Ferdinand  –  Take Me Out
7 Rad Cat  –  leave it all behind
8 MELVV  –  I'll Be There for You
9 demotapes  –  ghost
10 Juelz  –  Freaks & Geeks
11 Avicii  –  SOS (feat. Aloe Blacc)
12 Seven Lions  –  Not Even Love
13 Da Tweekaz  –  LA DI DA
14 Chance the Rapper  –  Buried Alive
15 Fred again..  –  places to be
16 Oliverse  –  Get High
17 Iniko  –  The King's Affirmation
18 J. Cole  –  Trae The Truth in Ibiza
19 John Summit  –  Go Back (feat. Julia Church)


This is how Spotipy reccomends we set up our API through the github. The issue with this is we need to create a new authentication for every different scope we want to call. 

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="400f5690f0bd413aab3a0b258e0c41f4",
                                               client_secret="03086e1b011c4cc18297e8c7c1dff61a",
                                               redirect_uri="http://localhost:7777/callback",
                                               scope="user-top-read"))

ranges = ['short_term', 'medium_term', 'long_term']

for sp_range in ranges:
    print("range:", sp_range)
    results1 = sp.current_user_top_tracks(time_range=sp_range, limit=10)
    for i, item in enumerate(results1['items']):
        print(i, item['name'], '//', item['artists'][0]['name'])
    print()


range: short_term
0 Under Control (feat. Hurts) // Calvin Harris
1 Too Sweet // Hozier
2 how2fly // ISOxo
3 Hymn for the Weekend - Seeb Remix // Coldplay
4 Wake Me Up // Avicii
5 Not Even Love // Seven Lions
6 leave it all behind // Rad Cat
7 Back Where I Belong (feat. Avicii) // Otto Knows
8 places to be // Fred again..
9 SOS (feat. Aloe Blacc) // Avicii

range: medium_term
0 Dawn // Eliminate
1 Cottage Gore // Subtronics
2 far away - Crankdat Remix // Mark Tuan
3 Huntin’ Wabbitz // J. Cole
4 Better Off (Alone, Pt. III) // Alan Walker
5 Wake Me Up // Avicii
6 F.Y.U. // Excision
7 Fever // J. Cole
8 Black Out Days - Subtronics Remix // Phantogram
9 King Of Nothing // BoyWithUke

range: long_term
0 Delilah (pull me out of this) // Fred again..
1 Danielle (smile on my face) // Fred again..
2 ten // Fred again..
3 Clara (the night is dark) // Fred again..
4 RATATA // Skrillex
5 King Of Nothing // BoyWithUke
6 Better Off Alone // Kai Wachi
7 Dawn // Eliminate
8 Cottage Gore // Subtronics
9

 What we need to accomplish now is to extract different features from my personal spotify data in order to determine what characteristics im drawn towards when creating my model. Additioanlly what ive noticed is that I need to create a new authorization scope each time I went to extract different types of datas instead i can 

 What we need to accomplish is to authenticate with the combined scope. This ensures that the client is authorized for all the required operations.

In [4]:
# Define all necessary scopes
scopes = [
    'user-top-read',
    'playlist-modify-private',
    'playlist-modify-public'
]

scope = ' '.join(scopes)

# Set up Spotify API client with multiple scopes
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id='400f5690f0bd413aab3a0b258e0c41f4',
    client_secret='03086e1b011c4cc18297e8c7c1dff61a',
    redirect_uri='http://localhost:7777/callback',
    scope=scope
))

In [5]:
cid = '400f5690f0bd413aab3a0b258e0c41f4'
secret = '03086e1b011c4cc18297e8c7c1dff61a'
redirect_uri='http://localhost:7777/callback'
scope ='user-top-read playlist-modify-private playlist-modify-public'

token = util.prompt_for_user_token(scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token")

The key above dosent print 

In [6]:
# Getting features for each song
def fetch_audio_features(sp, df):
    playlist = df[['track_id','track_name']] 
    index = 0
    audio_features = []
    
    # Make the API request
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    # Create an empty list to feed in different charactieritcs of the tracks
    features_list = []
    #Create keys-values of empty lists inside nested dictionary for album
    for features in audio_features:
        features_list.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness'],
                              features['mode']
                             ])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'acousticness', 'energy','tempo', 
                                                             'instrumentalness', 'loudness', 'liveness','duration_ms', 'key',
                                                             'valence', 'speechiness', 'mode'])
    
    # Create the final df, using the 'track_id' as index for future reference
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features

In [7]:
playlists = sp.user_playlists('spotify')
spotify_playlist_ids = []
while playlists:
    for i, playlist in enumerate(playlists['items']):
        spotify_playlist_ids.append(playlist['uri'][-22:])
    if playlists['next']:       
        playlists = sp.next(playlists)  
    else:
      playlists = None
spotify_playlist_ids[:20]

['37i9dQZF1DXcBWIGoYBM5M',
 '37i9dQZF1DX0XUsuxWHRQd',
 '37i9dQZF1DX1lVhptIYRda',
 '37i9dQZF1DX10zKzsJ2jva',
 '37i9dQZF1DX4JAvHpjipBk',
 '37i9dQZF1DX4sWSpwq3LiO',
 '37i9dQZF1DX4SBhb3fqCJd',
 '37i9dQZF1DWXRqgorJj26U',
 '37i9dQZF1DX4dyzvuaRJ0n',
 '37i9dQZF1DXcF6B6QPhFDv',
 '37i9dQZF1DWXJfnUiYjUKT',
 '37i9dQZF1DXcRXFNfZr7Tp',
 '37i9dQZF1DX4o1oenSJRJd',
 '37i9dQZF1DXbTxeAdrVG2l',
 '37i9dQZF1DX4UtSsGT1Sbe',
 '37i9dQZF1DWTJ7xPn4vNaz',
 '37i9dQZF1DXaKIA8E7WcJj',
 '37i9dQZF1DWSV3Tk4GO2fq',
 '37i9dQZF1DWTwnEm1IYyoj',
 '37i9dQZF1DX2A29LI7xHn1']

In [9]:
len(spotify_playlist_ids)

1027

In [10]:
def getTrackIDs(playlist_id):
    playlist = sp.user_playlist('spotify', playlist_id)
    for item in playlist['tracks']['items'][:50]:
        track = item.get('track')
        if track is not None:
            ids.append(track.get('id'))





In [11]:
# Creating a function get features of each track from track id
def getTrackFeatures(track_id):
  meta = sp.track(track_id)
  features = sp.audio_features(track_id)

  # meta
  track_id = track_id
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [track_id, name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  return track

In [12]:
ids = []
for x in spotify_playlist_ids[:200]:
    getTrackIDs(x)

print(ids[:5])


['5N3hjp1WNayUPZrA8kJmJP', '6dOtVTDdiauQNBQEDOtlAB', '6AI3ezQ4o3HUoP6Dhudph3', '0WbMK4wrZ1wFSty9F7FCgu', '2FQrifJ1N335Ljm3TjTVVf']


In [15]:
print(len(ids))

9823


In [16]:
# Function to split the list into n sublists
def split_list(input_list, n):
    k, m = divmod(len(input_list), n)
    return [input_list[i*k + min(i, m):(i+1)*k + min(i+1, m)] for i in range(n)]

# Splitting the list into 99 sublists
sublists = split_list(ids, 130)

# Checking the length of the sublists
for i, sublist in enumerate(sublists):
    print(f"Sublist {i+1} length: {len(sublist)}")

# Optional: If you need the sublists as separate variables
sublist_vars = [f'ids_{i+1}' for i in range(99)]
for var_name, sublist in zip(sublist_vars, sublists):
    globals()[var_name] = sublist

# Example: Accessing the first sublist
print(ids_1)
print(ids_2)



Sublist 1 length: 76
Sublist 2 length: 76
Sublist 3 length: 76
Sublist 4 length: 76
Sublist 5 length: 76
Sublist 6 length: 76
Sublist 7 length: 76
Sublist 8 length: 76
Sublist 9 length: 76
Sublist 10 length: 76
Sublist 11 length: 76
Sublist 12 length: 76
Sublist 13 length: 76
Sublist 14 length: 76
Sublist 15 length: 76
Sublist 16 length: 76
Sublist 17 length: 76
Sublist 18 length: 76
Sublist 19 length: 76
Sublist 20 length: 76
Sublist 21 length: 76
Sublist 22 length: 76
Sublist 23 length: 76
Sublist 24 length: 76
Sublist 25 length: 76
Sublist 26 length: 76
Sublist 27 length: 76
Sublist 28 length: 76
Sublist 29 length: 76
Sublist 30 length: 76
Sublist 31 length: 76
Sublist 32 length: 76
Sublist 33 length: 76
Sublist 34 length: 76
Sublist 35 length: 76
Sublist 36 length: 76
Sublist 37 length: 76
Sublist 38 length: 76
Sublist 39 length: 76
Sublist 40 length: 76
Sublist 41 length: 76
Sublist 42 length: 76
Sublist 43 length: 76
Sublist 44 length: 76
Sublist 45 length: 76
Sublist 46 length: 

The issue currently i am having is when loop over track ids to get audio features for each track I am being rate limited by spotify API . my solution for this is to take my list of ids and split it into seperate list of 100 items because 

In [17]:
tracks = []
for i in range(len(ids_1)):
    try:  
        track = getTrackFeatures(ids[i])
        tracks.append(track)
    except:
        pass

# create dataset
df = pd.DataFrame(tracks, columns = ['track_id', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df.head()

Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached


after trying multipleways to get pass rate limiting including developing a backoff-retry strategy ive decided to do some reaserach on different ways to scrape this data. i found SpotiFile which is a scrip that allows users to more easily scrape spotify playlist,albums and artist. After continued research ive concluded that both SpotifyScraper and the above API scraping methods have depreciated due to new spotify developer guidelines (section 5). "'scraping', whether manually or by automated means, or otherwise using any automated means (including bots, scrapers, and spiders), to view, access or collect information"